## 5-5. 데이터 전처리 및 Pre-Trained Embedding Vector를 이용한 Vocabulary생성

In [1]:
! nvidia-smi

Thu Feb 23 17:39:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 25%   35C    P0    N/A /  30W |   1924MiB /  2048MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install transformers

In [3]:
import re
import sys
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data
from torchtext import datasets

from transformers import BertTokenizer, BertModel

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
sentence = "My dog is cute. He likes playing. I bought a  pet food for him"
# sentence = '나는 책상 위에 사과를 먹었다. 알고 보니 그 사과는 Jason 것이었다. 그래서 Jason에게 사과를 했다'
print(tokenizer.tokenize(sentence))

['my', 'dog', 'is', 'cute', '.', 'he', 'likes', 'playing', '.', 'i', 'bought', 'a', 'pet', 'food', 'for', 'him']


In [6]:
len(tokenizer.vocab)

30522

In [7]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)
#BERT사용하려면 IMDb데이터를 BERT가 학습됐던 input과 동일하게 맞춰야함.

512


In [8]:
def new_tokenizer(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [9]:
def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

def PreProc(list_sentence):
    return [tokenizer.convert_tokens_to_ids(PreProcessingText(x)) for x in list_sentence]

In [10]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = new_tokenizer,
                  preprocessing = PreProc,
                  init_token = tokenizer.cls_token_id,
                  eos_token = tokenizer.sep_token_id,
                  pad_token = tokenizer.pad_token_id,
                  unk_token = tokenizer.unk_token_id)

LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
LABEL.build_vocab(train_data)
train_data, valid_data = train_data.split(random_state = random.seed(0), split_ratio=0.8)

## Reading Data(IMDb data)

In [11]:
# Data Length
print(f'Train Data Length : {len(train_data.examples)}') #data.examples : 데이터의 개수 확인
print(f'Test Data Length : {len(test_data.examples)}')

Train Data Length : 20000
Test Data Length : 25000


In [12]:
# Data Fields
train_data.fields

{'text': <torchtext.data.field.Field at 0x16edb84d448>,
 'label': <torchtext.data.field.LabelField at 0x16edb84c988>}

In [13]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(tokenizer.convert_ids_to_tokens(vars(train_data.examples[2])['text']))

---- Data Sample ----
Input : 
['another', 'in', 'a', 'long', 'line', 'of', 'flick', '##s', 'made', 'by', 'people', 'who', 'think', 'that', 'knowing', 'how', 'to', 'operate', 'a', 'camera', 'is', 'the', 'same', 'as', 'telling', 'a', 'story', '[UNK]', 'within', '15', 'minutes', '[UNK]', 'the', 'entire', 'premise', 'is', 'laid', 'out', 'in', 'just', 'a', 'few', 'lines', '[UNK]', 'so', 'there', 'is', 'absolutely', 'no', 'mystery', '[UNK]', 'which', 'eliminate', '##s', 'a', 'whole', 'face', '##t', 'of', 'the', 'suspense', '[UNK]', 'the', 'only', 'half', '[UNK]', 'way', 'competent', 'actor', 'is', 'killed', '10', 'minutes', 'into', 'the', 'film', '[UNK]', 'so', 'we', "'", 're', 'left', 'with', 'stupid', 'characters', 'running', 'around', 'doing', 'stupid', 'things', '[UNK]', 'low', 'budget', 'films', 'can', "'", 't', 'afford', 'expensive', 'special', 'effects', '[UNK]', 'so', 'the', 'c', '##gi', 'portions', 'are', 'un', '##sur', '##pr', '##ising', '##ly', 'un', '##im', '##pressive', '[UNK]'

## Pre-processing Data

## making vocab & setting embedding

In [14]:
# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
    print('\t', k, v)

Label Size : 2
Lable Examples : 
	 neg 0
	 pos 1


## spliting validation data & making data iterator

In [15]:
model_config = {}

model_config['batch_size'] = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=model_config['batch_size'],
    device=device)

In [16]:
# Check batch data
sample_for_check = next(iter(train_iterator))
print(sample_for_check)
print(sample_for_check.text)
print(sample_for_check.label)


[torchtext.data.batch.Batch of size 8]
	[.text]:[torch.cuda.LongTensor of size 8x512 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 8 (GPU 0)]
tensor([[ 101, 5752, 7104,  ...,    0,    0,    0],
        [ 101,  100, 8334,  ...,    0,    0,    0],
        [ 101, 1053,  100,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 2143,  ...,    0,    0,    0],
        [ 101, 4931, 2065,  ...,    0,    0,    0],
        [ 101, 2023, 3185,  ...,    0,    0,    0]], device='cuda:0')
tensor([1., 1., 1., 0., 0., 0., 0., 1.], device='cuda:0')


## Pytorch에서 RNN, LSTM, GRU을 이용한 모델 만들기

## Making Model

In [17]:
bert = BertModel.from_pretrained('bert-base-uncased')

model_config['emb_dim'] = bert.config.to_dict()['hidden_size']

print(model_config['emb_dim'])

TypeError: register_buffer() got an unexpected keyword argument 'persistent'

In [ ]:
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()
        self.bert = bert
        self.fc = nn.Linear(model_config['emb_dim'],
                            model_config['output_dim'])
        
    def forward(self, x):
        pooled_cls_output = self.bert(x)[1]
        return self.fc(pooled_cls_output)

## training

In [19]:
def train(model, iterator, optimizer, loss_fn, idx_epoch, **model_params):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() 
    batch_size = model_params['batch_size']

    for idx, batch in enumerate(iterator):
        
        # Initializing
        optimizer.zero_grad()
        
        # Forward 
        predictions = model(batch.text).squeeze()
        loss = loss_fn(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)
        
        sys.stdout.write(
                    "\r" + f"[Train] Epoch : {idx_epoch:^3}"\
                    f"[{(idx + 1) * batch_size} / {len(iterator) * batch_size} ({100. * (idx + 1) / len(iterator) :.4}%)]"\
                    f"  Loss: {loss.item():.4}"\
                    f"  Acc : {acc.item():.4}"\
                    )

        # Backward 
        loss.backward()
        optimizer.step()
        
        # Update Epoch Performance
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss/len(iterator) , epoch_acc/len(iterator) 

In [20]:
def evaluate(model, iterator, loss_fn, idx_epoch, **model_params):
    
    epoch_loss = 0
    epoch_acc = 0
    
    batch_size = model_params['batch_size']
    
    # evaluation mode
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(iterator):
            predictions = model(batch.text).squeeze()
            loss = loss_fn(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

            sys.stdout.write(
                    "\r" + f"[Eval] Epoch : {idx_epoch:^3}"\
                    f"[{(idx + 1) * batch_size} / {len(iterator) * batch_size} ({100. * (idx + 1) / len(iterator) :.4}%)]"\
                    f"  Loss: {loss.item():.4}"\
                    f"  Acc : {acc.item():.4}"\
                    )
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

## bi-RNN

In [21]:
model_config.update(dict(output_dim = 1))

def binary_accuracy(preds, y):
    # rounded_preds = torch.argmax(preds, axis=1) 
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum()/len(correct)
    return acc


model = SentenceClassification(**model_config)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = nn.BCEWithLogitsLoss().to(device)
model = model.to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

In [ ]:
N_EPOCH = 4

best_valid_loss = float('inf')
model_name = "BERT"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    print('')
    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn, epoch, **model_config)
    print('')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')
    # print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Model is saved at {epoch}-epoch')

---------------------------------
Model name : bi-RNN_
---------------------------------
[Train] Epoch :  0 [20010 / 20010 (100.0%)]  Loss: 0.6795  Acc : 0.5333
	 Saved at 0-epoch
	 Epoch : 0 | Train Loss : 0.6333 | Train Acc : 0.6351
	 Epoch : 0 | Valid Loss : 0.6239 | Valid Acc : 0.6356
[Train] Epoch :  1 [20010 / 20010 (100.0%)]  Loss: 0.7517  Acc : 0.6333
	 Epoch : 1 | Train Loss : 0.6192 | Train Acc : 0.6449
	 Epoch : 1 | Valid Loss : 0.6446 | Valid Acc : 0.6201
[Train] Epoch :  2 [20010 / 20010 (100.0%)]  Loss: 0.6333  Acc : 0.7333
	 Epoch : 2 | Train Loss : 0.6027 | Train Acc : 0.6623
	 Epoch : 2 | Valid Loss : 0.6562 | Valid Acc : 0.6108
[Train] Epoch :  3 [14940 / 20010 (74.66%)]  Loss: 0.6388  Acc : 0.7677

In [ ]:
# Test set
# model.load_state_dict(torch.load(f'./{model_name}.pt'))
epoch = 0
test_loss, test_acc = evaluate(model, test_iterator, loss_fn, epoch, **model_config)
print('')
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
epoch = 0
test_loss, test_acc = evaluate(model, test_iterator, loss_fn, epoch, **model_config)
print('')
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

## bi-LSTM

In [ ]:
model_config['model_type'] = 'LSTM'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

## bi-GRU

In [ ]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss().to(device)

In [ ]:
N_EPOCH = 5

best_valid_loss = float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
test_loss, test_acc = evaluate(model, test_iterator, loss_fn)
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')

## inference

In [ ]:
model_config['model_type'] = 'GRU'
model = SentenceClassification(**model_config).to(device)
model.load_state_dict(torch.load(f"./{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}.pt"))

In [ ]:
def predict_sentiment(model, sentence):
    model.eval()
    indexed = TEXT.numericalize(TEXT.pad([TEXT.tokenize(PreProcessingText(sentence))]))
    input_data = torch.LongTensor(indexed).to(device)
    prediction = torch.sigmoid(model(input_data))
    return prediction.item()

In [ ]:
test_sentence = 'this movie is FUN'
predict_sentiment(model = model, sentence = test_sentence)